In [1]:
import datacube
import numpy as np

# Create a query object
lat, lon = 10.821, 28.518
buffer = 1

query = {
    'time': ('2020-10-01', '2020-10-5'),
    'x': (lon - buffer, lon + buffer),
    'y': (lat + buffer, lat - buffer),
    'output_crs': 'epsg:32635',
    'resolution':(-20,20),
}

In [2]:
dc = datacube.Datacube(app="s2-l1c-eu")
ds = dc.load(product='s2a_level1c_granule',
             dask_chunks={},
             **query)
ds = ds.isel(time=0)
ds

<xarray.Dataset>
Dimensions:      (x: 10989, y: 11117)
Coordinates:
    time         datetime64[ns] 2020-10-02T08:37:32.329132
  * y            (y) float64 1.308e+06 1.308e+06 ... 1.086e+06 1.086e+06
  * x            (x) float64 5.564e+05 5.564e+05 ... 7.762e+05 7.762e+05
    spatial_ref  int32 32635
Data variables:
    B01          (y, x) uint16 dask.array<chunksize=(11117, 10989), meta=np.ndarray>
    B02          (y, x) uint16 dask.array<chunksize=(11117, 10989), meta=np.ndarray>
    B03          (y, x) uint16 dask.array<chunksize=(11117, 10989), meta=np.ndarray>
    B04          (y, x) uint16 dask.array<chunksize=(11117, 10989), meta=np.ndarray>
    B05          (y, x) uint16 dask.array<chunksize=(11117, 10989), meta=np.ndarray>
    B06          (y, x) uint16 dask.array<chunksize=(11117, 10989), meta=np.ndarray>
    B07          (y, x) uint16 dask.array<chunksize=(11117, 10989), meta=np.ndarray>
    B08          (y, x) uint16 dask.array<chunksize=(11117, 10989), meta=np.ndarray>
    B8A          (y, x) uint16 dask.array<chunksize=(11117, 10989), meta=np.ndarray>
    B09          (y, x) uint16 dask.array<chunksize=(11117, 10989), meta=np.ndarray>
    B10          (y, x) uint16 dask.array<chunksize=(11117, 10989), meta=np.ndarray>
    B11          (y, x) uint16 dask.array<chunksize=(11117, 10989), meta=np.ndarray>
    B12          (y, x) uint16 dask.array<chunksize=(11117, 10989), meta=np.ndarray>
Attributes:
    crs:           epsg:32635
    grid_mapping:  spatial_ref

In [3]:
arr = ds.to_array().values

In [4]:
# convert to shape n_images x n x m x 13 for all_bands=True
print(arr.shape)
arr = np.moveaxis(arr, 0, -1)
print(arr.shape)
arr = np.expand_dims(arr, axis=0)
print(arr.shape)

(13, 11117, 10989)
(11117, 10989, 13)
(1, 11117, 10989, 13)


In [5]:
from s2cloudless import S2PixelCloudDetector

cloud_detector = S2PixelCloudDetector(threshold=0.4, average_over=1, dilation_size=1, all_bands=True)

In [6]:
#cloud_masks = cloud_detector.get_cloud_masks(arr)
cloud_masks = cloud_detector.get_cloud_probability_maps(arr)

In [7]:
print(cloud_masks.shape)
cloud_masks = np.squeeze(cloud_masks)
print(cloud_masks.shape)
print(cloud_masks)

(1, 11117, 10989)
(11117, 10989)
[[0.00011124 0.00011124 0.00011124 ... 0.00011124 0.00011124 0.00011124]
 [0.00011124 0.00011124 0.00011124 ... 0.00011124 0.00011124 0.00011124]
 [0.00011124 0.00011124 0.00011124 ... 0.00011124 0.00011124 0.00011124]
 ...
 [0.00011124 0.00011124 0.00011124 ... 0.00011124 0.00011124 0.00011124]
 [0.00011124 0.00011124 0.00011124 ... 0.00011124 0.00011124 0.00011124]
 [0.00011124 0.00011124 0.00011124 ... 0.00011124 0.00011124 0.00011124]]


In [8]:
import gdal
gdal.UseExceptions()
from typing import List


def array_to_geotiff_multiband(fname: str, data: List[np.ndarray],
                               geo_transform: tuple, projection: str,
                               nodata_val=0, dtype=gdal.GDT_Float32):
    """ Create a multiband GeoTIFF file with data from an array.    
    fname : output geotiff file path including extension
    data : list of numpy arrays    
    geo_transform : Geotransform for output raster; e.g.
    "(upleft_x, x_size, x_rotation, upleft_y, y_rotation, y_size)"
    projection : WKT projection for output raster
    nodata_val : Value to convert to nodata in the output raster; default 0
    dtype : gdal dtype object, optional
        Optionally set the dtype of the output raster; can be 
        useful when exporting an array of float or integer values. """
    driver = gdal.GetDriverByName('GTiff')
    rows, cols = data[0].shape  # Create raster of given size and projection
    dataset = driver.Create(fname, cols, rows, len(data), dtype)
    dataset.SetGeoTransform(geo_transform)
    dataset.SetProjection(projection)
    for idx, d in enumerate(data):
        band = dataset.GetRasterBand(idx + 1)
        band.WriteArray(d)
        band.SetNoDataValue(nodata_val)
    dataset = None  # Close %%file

In [9]:
geotrans = ds.geobox.transform.to_gdal()
prj = ds.geobox.crs.wkt
values = [cloud_masks]
array_to_geotiff_multiband('/notebooks/cloud_prob.tif', values, geotrans, prj)

In [ ]:
bands = ['B04', 'B03', 'B02']
values = [ds[band].values for band in bands]
array_to_geotiff_multiband('/notebooks/rgb.tif', values, geotrans, prj)

In [15]:
ds2 = dc.load(product='s2_l2a',
             dask_chunks={},
             **query)
ds2 = ds2.isel(time=0)
ds2

<xarray.Dataset>
Dimensions:      (x: 10989, y: 11117)
Coordinates:
    time         datetime64[ns] 2020-10-02T08:37:00
  * y            (y) float64 1.308e+06 1.308e+06 ... 1.086e+06 1.086e+06
  * x            (x) float64 5.564e+05 5.564e+05 ... 7.762e+05 7.762e+05
    spatial_ref  int32 32635
Data variables:
    B01          (y, x) uint16 dask.array<chunksize=(11117, 10989), meta=np.ndarray>
    B02          (y, x) uint16 dask.array<chunksize=(11117, 10989), meta=np.ndarray>
    B03          (y, x) uint16 dask.array<chunksize=(11117, 10989), meta=np.ndarray>
    B04          (y, x) uint16 dask.array<chunksize=(11117, 10989), meta=np.ndarray>
    B05          (y, x) uint16 dask.array<chunksize=(11117, 10989), meta=np.ndarray>
    B06          (y, x) uint16 dask.array<chunksize=(11117, 10989), meta=np.ndarray>
    B07          (y, x) uint16 dask.array<chunksize=(11117, 10989), meta=np.ndarray>
    B08          (y, x) uint16 dask.array<chunksize=(11117, 10989), meta=np.ndarray>
    B8A          (y, x) uint16 dask.array<chunksize=(11117, 10989), meta=np.ndarray>
    B09          (y, x) uint16 dask.array<chunksize=(11117, 10989), meta=np.ndarray>
    B11          (y, x) uint16 dask.array<chunksize=(11117, 10989), meta=np.ndarray>
    B12          (y, x) uint16 dask.array<chunksize=(11117, 10989), meta=np.ndarray>
    SCL          (y, x) uint8 dask.array<chunksize=(11117, 10989), meta=np.ndarray>
    AOT          (y, x) uint16 dask.array<chunksize=(11117, 10989), meta=np.ndarray>
    WVP          (y, x) uint16 dask.array<chunksize=(11117, 10989), meta=np.ndarray>
Attributes:
    crs:           epsg:32635
    grid_mapping:  spatial_ref

In [16]:
array_to_geotiff_multiband('/notebooks/scl.tif', [ds2.SCL.values], geotrans, prj)